# _2_00_patent_IE
- Author: Tommy Xie (TommyXie_@outlook.com)
- Oct 2024
- This script utilises an LLM via official API from openai (you would need a valid API Key for this) to extract fields that are useful in different ways for downstream processing. Crucial ones are names and state,
- Feel free to neglect occupation extraction. They were first brought up to evaluate matching outcomes. For usage as a feature for matching, circular argument might be resulted (since the target for this data work (scripts starting with \_2\_*\_) is to bring inventors and their occupations together from two datasets).

In [1]:
# !pip install -q pandas
# !pip install dask
# !pip install pyarrow
# !pip install fastparquet phonetics fuzzywuzzy jellyfish -q
# !pip install seaborn -q 
# !pip install matplotlib -q 

import pandas as pd
# !pip install recordlinkage
import recordlinkage as rl
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import dask.dataframe as dd
import dask
from recordlinkage.preprocessing import clean, phonetic
# get cpu number
import multiprocessing
try:
    import inspect

except:
    !pip install inspect
    import inspect

multiprocessing.cpu_count()

# garbage collection
import gc
gc.collect()

# get RAM
!cat /proc/meminfo

MemTotal:       32138704 kB
MemFree:        27133164 kB
MemAvailable:   30918692 kB
Buffers:            2704 kB
Cached:          4088900 kB
SwapCached:            0 kB
Active:          1771460 kB
Inactive:        2912828 kB
Active(anon):        712 kB
Inactive(anon):   592888 kB
Active(file):    1770748 kB
Inactive(file):  2319940 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:                 8 kB
Writeback:             0 kB
AnonPages:        592880 kB
Mapped:           299944 kB
Shmem:               916 kB
KReclaimable:     111320 kB
Slab:             166068 kB
SReclaimable:     111320 kB
SUnreclaim:        54748 kB
KernelStack:        5600 kB
PageTables:        10892 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    16069352 kB
Committed_AS:    2431176 kB
VmallocTotal:   34359738367 kB
VmallocUsed:       13996 kB
VmallocChunk:          0 kB
Percpu:          

In [2]:
import spacy
import re
from spacy.tokens import DocBin
from fuzzywuzzy import fuzz
import pandas as pd

# import Doc
from spacy.tokens import Doc

def clean_cols(df):
    '''
    use snaky naming
    '''
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    return df

def get_doc_bin_custom(data, nlp):
    db = DocBin()
    for text, annotations in data:
        doc = Doc(nlp.vocab, words=text.split())
        # print("Text:", text)
        # print("Annotations:", annotations)
        ents = []
        for start, end, label in annotations['entities']:
            span = doc.char_span(start, end, label=label)
            if span is not None:
                print("Span:", span)
                ents.append(span)
            else:
                print("Warning: No span found for", start, end, label)
        doc.ents = ents
        db.add(doc)
    return db

stop_words = ['SPECIFICATION', 'PROVISIONAL', 'PROVISION', 'COMPLETE', '"']

def clean_text(df):
    '''
    clean text
    '''
    # use RE to replace between "IN THE" and "OF"
    df['description'] = df['description'].apply(lambda x: re.sub(r'IN THE \w+ OF', '', x))
    for stop_word in stop_words:
        df.description = df.description.str.replace(stop_word, '')
    df.description = df.description.str.replace('(', '').str.replace(')', '').str.replace(',', ' ').str.replace('  ', ' ').str.replace('  ', ' ').str.strip()
    
    # the same for address_1  address_2 address_3
    for col in ['address_1', 'address_2', 'address_3']:
        if col not in df.columns:
            continue
        # make it string
        df[col] = df[col].astype(str)
        # use re
        df[col] = df[col].apply(lambda x: re.sub(r'IN THE \w+ OF', '', x))
        df[col] = df[col].str.replace('(', '').str.replace(')', '').str.replace(',', ' ').str.replace('  ', ' ').str.replace('  ', ' ').str.strip()

    return df

def find_best_match_substring(search_text, query):
    words = search_text.split()
    best_match = None
    highest_score = 0
    best_start = None
    best_end = None

    # Iterate over all possible substrings
    for start in range(len(words)):
        for end in range(start, min(len(words), start + 10)):  # Limiting the length of substring
            substring = ' '.join(words[start:end+1])
            len_diff = abs(len(query) - len(substring))
            len_resemblance = 1 / (len_diff + 1)  # Normalized length difference

            score = fuzz.ratio(substring, query) * len_resemblance

            if score > highest_score:
                highest_score = score
                best_match = substring
                best_start = start
                best_end = end

            if highest_score > 95:  # Threshold for a good match
                break

    # Convert word indices to character indices
    if best_match is None:
        return None, None, None, None
    
    start_idx = len(' '.join(words[:best_start])) + (1 if best_start > 0 else 0)
    end_idx = len(' '.join(words[:best_end + 1]))

    return start_idx, end_idx, best_match, highest_score

/users/xiet13/.local/lib/python3.11/site-packages/torch/cuda/__init__.py:611: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/users/xiet13/.local/lib/python3.11/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
import pandas as pd

# Load the Excel file
excel_file = pd.ExcelFile("../ori_data/US_patent/Legend.xlsx")
excel_file.sheet_names # ['US1880a_CITY', 'State_County', 'US1880A_METDIST', 'METROUS']

# legend
US1880a_CITY = pd.read_excel("../ori_data/US_patent/Legend.xlsx", sheet_name = "US1880a_CITY")
State_County = pd.read_excel("../ori_data/US_patent/Legend.xlsx", sheet_name = "State_County")
US1880A_METDIST = pd.read_excel("../ori_data/US_patent/Legend.xlsx", sheet_name = "US1880A_METDIST")
METROUS = pd.read_excel("../ori_data/US_patent/Legend.xlsx", sheet_name = "METROUS")

In [4]:
# patent = pd.read_csv("../ori_data/US_patent/PatentsDataUS_Clean.csv")
patent = pd.read_feather("../int_data/PatentsDataUS_Clean.feather")
patent = patent.reset_index().rename(columns = {"Description":"description"})
# drop unnecessary columns
patent.drop(columns = ['inventorsnames1',	'inventorsnames2',	'inventorsnames3',	'inventorsnames4', 'inventorsnames5', 'inventorsnames6'], inplace = True) # these columns are sparse, and all their info are covered in the LLM output
patent

,index,publicationnumber,X,Publication.date,Title,description,year
0,0,US100000A,1,22/02/1870,None,"No. 100,000. J. ANDERSON. Horse Sun-Bonnet. Pa...",1870
1,1,US100001A,2,22/02/1870,None,J. ARRINGTON. Walking Planter. N. PETERS. PHOT...,1870
2,2,US100002A,3,22/02/1870,None,"United States Patent Office. HENRY BARTH, OF C...",1870
3,3,US100003A,4,22/02/1870,None,"No. 100,003. PROCESS OF AND APPARATUS PATENTED...",1870
4,4,US100004A,5,22/02/1870,None,"N. PETERS, PHOTO-LITHOGRAPHER. WASHINGTON. 0 C...",1870
...,...,...,...,...,...,...,...
309030,309030,USRE8396E,18494,03/09/1878,None,D. H. PEARCE &amp; A. W. TAFT. Assignor of one...,1878
309031,309031,USRE8666E,18495,08/04/1879,None,"Sheets—Sheet 1. L. P. JUVET, Time-Globe. No. 8...",1879
309032,309032,USRE9467E,18497,16/11/1880,None,B. FRESE. Roller Abstractor for Watches. No. 9...,1880
309033,309033,USRE9656E,18498,12/04/1881,None,Sheets—Sheet i. A. E. HOTCHKISS. Clock Movemen...,1881


In [5]:
patent.shape[0]

309035

In [6]:
from dask.distributed import LocalCluster
cluster = LocalCluster(n_workers=8, threads_per_worker=4)

In [7]:
sample_size = 309035
# sample_size = 3000

# sub_patents = patent.sample(sample_size, random_state = 42).reset_index(drop = True)
sub_patents = patent

In [8]:
(patent == sub_patents).all()

index                 True
publicationnumber     True
X                     True
Publication.date      True
Title                False
description           True
year                  True
dtype: bool

In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv)
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

client = openai.Client()

### API Usage

In [10]:
import openai
import requests
from datetime import datetime, timedelta

# Set your API key
# openai.api_key = 'your-api-key-here'

# Function to fetch usage data (assuming OpenAI provides an endpoint for this)
def fetch_usage_data():
    headers = {
        "Authorization": f"Bearer {openai.api_key}",
    }
    response = requests.get("https://api.openai.com/v1/usage", headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch data: {response.status_code}")
        return None

# Example usage
usage_data = fetch_usage_data()
if usage_data:
    print("Usage Data:", usage_data)


Failed to fetch data: 401


## IE with LLM

In [11]:
# truncate text to first n chars
TRUNCATING_AT = 600
sub_patents.description = sub_patents.description.apply(lambda x: x[:TRUNCATING_AT])
sub_patents.description.apply(len)

0         600
1         600
2         600
3         600
4         600
         ... 
309030    600
309031    600
309032    600
309033    600
309034    600
Name: description, Length: 309035, dtype: int64

In [12]:
sub_patents.description.iloc[0]

'No. 100,000. J. ANDERSON. Horse Sun-Bonnet. Patented Feb. 22, 1870. N. PETERS. Photo-Lithographer. Wa.hington, D. CUnited States patent (^ffteu JOHN ANDERSON, OF BROOKLYN, NEW YORK. Letters Patent No. 100,000, dated February 22,1870. IMPROVED SUN-BONNET FOR HORSES. The Schedule referred to in these Letters Patent and making part of the same. To all whom it may concern : Be it known that I, John Andeeson, of Brooklyn, Kings County, and State of New York, have invented a new and useful Improvement in Shields or Sun-Bonnets for Horses;, and do hereby declare that the following is a general descri'

In [13]:
# import copy
# _patents_copy = copy.copy(patent)
# _patents_copy.description = _patents_copy.description.apply(lambda x: x[:TRUNCATING_AT])
# _patents_copy.shape

In [14]:
try:
    import tiktoken
except:
    !pip install tiktoken
    import tiktoken

# Step 1: Choose the tokenizer corresponding to your model
# For example, if you are using GPT-4:
tokenizer = tiktoken.get_encoding("cl100k_base")

# Step 2: Define your text
text = sub_patents.description.tolist()
# text = _patents_copy.description.tolist()

# Step 3: Tokenize the text
tokens = tokenizer.encode(''.join(text))

# Step 4: Count the number of tokens
token_count = len(tokens)

print(f"Number of tokens: {token_count}")
# $3.85

model = 'gpt-4o-mini'

def calculate_cost(input_tokens, output_tokens, model='gpt-4o-2024-08-06'):
    # Define the rates for GPT-4o and GPT-4o Mini
    if model == 'gpt-4o-2024-08-06':
        input_rate_per_1000 = 2.50 / 1000  # $5 per million tokens = $0.005 per 1000 tokens
        output_rate_per_1000 = 10.00 / 1000  # $15 per million tokens = $0.015 per 1000 tokens
    elif model == 'gpt-4o-mini':
        input_rate_per_1000 = 0.15 / 1000  # $0.15 per million tokens = $0.00015 per 1000 tokens
        output_rate_per_1000 = 0.60 / 1000  # $0.60 per million tokens = $0.00060 per 1000 tokens
    else:
        raise ValueError("Invalid model name. Choose either 'gpt-4o-2024-08-06' or 'gpt-4o-mini'.")

    # Calculate cost for input tokens
    input_cost = (input_tokens / 1000) * input_rate_per_1000
    
    # Calculate cost for output tokens
    output_cost = (output_tokens / 1000) * output_rate_per_1000
    
    # Total cost
    total_cost = input_cost + output_cost
    
    return total_cost
    
input_tokens = token_count
output_tokens = input_tokens / 6 # assume they are the same

cost = calculate_cost(input_tokens, output_tokens, model = model)
print(f"tokens for {input_tokens + output_tokens} tokens using {model}: Total cost: ${cost:.4f}")
# 

Number of tokens: 49137872
tokens for 57327517.333333336 tokens using gpt-4o-mini: Total cost: $12.2845


- TRUNCATING_AT = 600  

Number of tokens: 49137872    
tokens for 57328412.166666664 tokens using gpt-4o-mini: Total cost: $12.2847



In [ ]:
try:
    import openai
except ImportError:
    !pip install openai
    import openai
import os

# client = openai.Client()

# TODO: add enum to response_format -> states
# model="gpt-4o-2024-08-06"
model="gpt-4o-mini"
prompt_flag = 'ver0_2'

def parse_patent(row, model = model):
    '''
     # Genrally, the model should avoid hallucination to maximum
        # Here, we want as more info about inventors as possible that matches with 19th century census features such as [occupation, industry] that might not be explictly in
        # the patent text, but could be inferred. So these two fields are generally missing so we assume all are inferred and therefore do not allow null values.
        # We also can design simple metrics to quantify to which degree these inferred personal features are hallucinations

    # Extra: sex is completely inferred and we want the model to infer as completely as possible
    '''
    response = client.chat.completions.create(
        model = model,
        messages=[
            {"role": "system", "content": '''
            You are an expert in information extraction. You have been asked to extract all information related to the name, occupation, and address for inventors in a patent document. The text are OCR results of historical patents in the US.
            
            Please adhere to the following instructions:
            1. Return a JSON object for each inventor following the response format specified. Ensure each inventor has their properties assigned correctly.
            2. Try to populate every propety for 'address'. This is in the format of 1880 US. For any field missing from the original patent description, just leave blank ''. If the address of an inventor is missing, or belongs to another inventor, do not guess or misassign.
            3. Summarise the text and then infer 'occupation' and 'industry' fields according to (Historical International Standard Classification of Occupations (HISCO) coding scheme)
            '''},
            {"role": "user", "content": row['description']},
        ],
           
         response_format={
                "type": "json_schema",
                "json_schema": {
                    "name": "patent_inventors",
                    "schema": {
                        "type": "object",
                        "properties": {
                            "inventors": {
                                "type": "array",
                                "items": {
                                    "type": "object",
                                    "properties": {
                                        "name": {
                                            "type": ["string", "null"],
                                            "description": "The name of the inventor. For example, 'Acle', 'Hempnall', 'Tibenham', 'Blakeney'. If you don't know, return null."
                                        },
                                        "sex":{
                                            "type": ["boolean", "null"],
                                            "description": "Sex inferred from the inventor's first name. 0 for male, 1 for female. For example, 'ANNIE', 'MARY', 'LYDIA' are female."
                                        },
                                        "occupation": {
                                            "type": ["string"],
                                            "description": "Summarise the text and then infer the occupation of the inventor (Historical International Standard Classification of Occupations (HISCO) coding scheme). For example, 'Industrial machinery or tools engineers', 'Engineering technicians nec', 'Ships officers nfs', 'Newsvendors'."
                                        },
                                        "industry": {
                                            "type": ["string"],
                                            "description": "The industry of the inventor's occupation belongs to (according to Industry 1950 basis, US). For example, 'Aircraft and parts', 'Retail florists', 'Postal service', 'Lady/Man of leisure', 'Common or General laborer'."
                                        },
                                            
                                        "address": {
                                            "type": "object",
                                            "properties": {
                                                "street": {
                                                    "type": ["string", "null"],
                                                    "description": "Street address of the inventor. More detailed than city. For example, Kennett Square, Elm Ave, Maple Drive, Oak Lane. If you don't know, just leave blank"
                                                },
                                                "city": {
                                                    "type": ["string", "null"],
                                                    "description": "City of the inventor. For example: ['Attleboro, MA', ' Cincinnati, OH', 'Long Island City, NY']"
                                                },
                                                "county": {
                                                    "type": ["string", "null"],
                                                    "description": "County of the inventor. More detailed than state, but less than city"
                                                },
                                                "metro":{
                                                    "type": ["string", "null"],
                                                    "description":" Metropolitan areas are counties or combinations of counties centering on a substantial urban area. For example: ['Dover, DE', '   Louisville, KY/IN'] . "
                                                },
                                                "state": {
                                                    "type": ["string", "null"],
                                                    "enum": [
                                                            "Alabama", "Illinois", "District of Columbia", "Maine", "Michigan",
                                                            "Iowa", "Missouri", "New York", "Ohio", "Pennsylvania", "Florida",
                                                            "Texas", "Wisconsin", "Kansas", "Maryland", "Georgia", "Indiana",
                                                            "Utah", "Massachusetts", "Kentucky", "Vermont", "North Carolina",
                                                            "Arizona", "Arkansas", "Montana", "Nebraska", "Minnesota",
                                                            "Virginia", "Oregon", "Nevada", "New Hampshire", "Idaho",
                                                            "Rhode Island", "California", "New Jersey", "Mississippi",
                                                            "South Carolina", "Louisiana", "Colorado", "Washington",
                                                            "West Virginia", "Tennessee", "Connecticut", "New Mexico",
                                                            "South Dakota", "North Dakota", "Delaware"
                                                        ],
                                                    "description": "State of the inventor."
                                                },
                                                "region": {
                                                    "type": ["string", "null"],
                                                    "description": "Census region and division of the inventor. For example: ['New England Division, 'Middle Atlantic Division', 'East North Central Division', 'West North Central Division', 'South Atlantic Division', 'East South Central Division', 'West South Central Division', 'Mountain Division',  'Pacific Divisio']."
                                                },
                                                "foreign": {
                                                    "type": ["boolean", "null"],
                                                    "description": "True indicates froeign to the US, otherwise False."
                                                },
                                                "full_address": {
                                                    "type": ["string", "null"],
                                                    "description": "The full address string."
                                                },
                                            },
                                            "required": ["street", "county", "city", "state", "region", "foreign", "full_address"],
                                            "additionalProperties": False
                                        }
                                    },
                                    "required": ["name", "occupation", "industry", "address"],
                                    "additionalProperties": False
                                }
                            }
                        },
                        "required": ["inventors"],
                        "additionalProperties": False
                    }
                }
            }
    )
    row['parsed_desc'] = response.choices[0].message.content
    # budget
    row['prompt_tokens'] = response.usage.prompt_tokens
    row['completion_tokens'] = response.usage.completion_tokens
    row['total_tokens'] = response.usage.total_tokens
    row['cost'] = calculate_cost(row['prompt_tokens'], row['completion_tokens'], model = model)
    return row

# for each row, ask chat gpt-3 to parse raw_title into region, road, type_of_work, starting_date, ending_date, duration
import concurrent.futures
from tqdm import tqdm
import pandas as pd
import multiprocessing as mp

cpu_count = mp.cpu_count()
print(cpu_count)
# workers = cpu_count - 2
workers = cpu_count

def parallel_apply_rows(data, func, workers=workers):
    with concurrent.futures.ThreadPoolExecutor(max_workers=workers) as executor:
        results = list(tqdm(executor.map(func, [row for _, row in data.iterrows()]), total=len(data)))
    return pd.DataFrame(results)


def parallel_apply(data, func, workers=workers, *args, **kwargs):
    """Apply a function in parallel with additional arguments, preserving order."""
    with concurrent.futures.ThreadPoolExecutor(max_workers=workers) as executor:
        # Submit tasks with their original index
        futures = {executor.submit(func, item, *args, **kwargs): i for i, item in enumerate(data)}
        
        # Collect results, ensuring they are ordered by their original index
        results = [None] * len(data)
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
            idx = futures[future]
            results[idx] = future.result()
    return results

# # Apply the parse_patent function in parallel
# sub_patents = parallel_apply_rows(sub_patents, parse_patent)

MODEL = "gpt-4o-mini"
BATCH_SIZE = 3000
OUTPUT_DIR = "../int_data/batch_processed"
# starting_index = 0
starting_index = 285000 # So we can start from where we stopped
os.makedirs(OUTPUT_DIR, exist_ok = True)
    
for i in range(starting_index, len(sub_patents), BATCH_SIZE):
    batch = sub_patents.iloc[i:i + BATCH_SIZE]
    batch = parallel_apply_rows(batch, parse_patent)
    batch.to_pickle(f"{OUTPUT_DIR}/batch_{i}_{i+BATCH_SIZE}.pickle")

print("Batch processing complete and data saved.")

8


 71%|███████   | 2136/3000 [04:31<02:04,  6.95it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

100%|██████████| 35/35 [00:05<00:00,  5.90it/s]

Batch processing complete and data saved.


In [16]:
!ls -lhta ../int_data/batch_processed/

total 305M
-rw-r--r-- 1 xiet13 cluster-users  40K Oct 21 22:57 batch_309000_312000.pickle
drwxr-xr-x 2 xiet13 cluster-users  14K Oct 21 22:57 .
-rw-r--r-- 1 xiet13 cluster-users 3.1M Oct 21 22:57 batch_306000_309000.pickle
-rw-r--r-- 1 xiet13 cluster-users 2.9M Oct 21 22:50 batch_303000_306000.pickle
-rw-r--r-- 1 xiet13 cluster-users 2.9M Oct 21 22:44 batch_300000_303000.pickle
-rw-r--r-- 1 xiet13 cluster-users 2.9M Oct 21 22:37 batch_297000_300000.pickle
-rw-r--r-- 1 xiet13 cluster-users 2.9M Oct 21 22:31 batch_294000_297000.pickle
-rw-r--r-- 1 xiet13 cluster-users 2.9M Oct 21 22:24 batch_291000_294000.pickle
-rw-r--r-- 1 xiet13 cluster-users 3.0M Oct 21 22:11 batch_288000_291000.pickle
-rw-r--r-- 1 xiet13 cluster-users 2.9M Oct 21 22:05 batch_285000_288000.pickle
-rw-r--r-- 1 xiet13 cluster-users 3.1M Oct 21 21:28 batch_282000_285000.pickle
-rw-r--r-- 1 xiet13 cluster-users 2.9M Oct 21 21:21 batch_279000_282000.pickle
-rw-r--r-- 1 xiet13 cluster-users 2.9M Oct 21 21:14 batch_276000_2

In [31]:
base_path = "../int_data/batch_processed/"
batch_files = []
for file in os.listdir(base_path):
    full_path = base_path + file
    # print(full_path)
    batch = pd.read_pickle(full_path)
    batch_files.append(batch)
sub_patents = pd.concat(batch_files, axis = 0).drop(columns = ["index"]).reset_index(drop = True)
sub_patents 

,publicationnumber,X,Publication.date,Title,description,year,parsed_desc,prompt_tokens,completion_tokens,total_tokens,cost
0,US343063A,47890,01/06/1886,None,"Application filed November 24, 1885. Serial No...",1886,"{""inventors"":[{""name"":""Charles F. Plumb"",""sex""...",1037,67,1104,0.000196
1,US343064A,47891,01/06/1886,None,"Application filed April 1,1886. Serial No. 197...",1886,"{""inventors"":[{""name"":""William F. Price"",""sex""...",1035,73,1108,0.000199
2,US343065A,47892,01/06/1886,None,"Application filed April 16, 1886. Serial No. 1...",1886,"{""inventors"":[{""name"":""Thomas B. Purves"",""sex""...",1033,78,1111,0.000202
3,US343066A,47893,01/06/1886,None,"Application filed February 23, 1886. Serial No...",1886,"{""inventors"":[{""name"":""William D. Ready"",""sex""...",1022,65,1087,0.000192
4,US343067A,47894,01/06/1886,None,"Application filed September 4, 1885. Serial No...",1886,"{""inventors"":[{""name"":""John F. Boeking"",""sex"":...",1036,61,1097,0.000192
...,...,...,...,...,...,...,...,...,...,...,...
309030,US215188A,50404,06/05/1879,None,"Μ, Β. WEBSTER. Loose Pulley. United States Pat...",1879,"{""inventors"":[{""name"":""Milo B. Webster"",""sex"":...",1049,70,1119,0.000199
309031,US215189A,50405,06/05/1879,None,J. D. WESTGATE. Edge-Trimming Tool for Boots a...,1879,"{""inventors"":[{""name"":""Joseph D. Westgate"",""se...",1071,70,1141,0.000203
309032,US215190A,50406,06/05/1879,None,"R. P. WILLIAMS. Derrick-Stake. No. 215,190. Pa...",1879,"{""inventors"":[{""name"":""R. P. WILLIAMS"",""sex"":0...",1070,55,1125,0.000193
309033,US215191A,50407,06/05/1879,None,"N. PETERS. PHOTO-LITHOGRAPHER. WASHINGTON, 0 C...",1879,"{""inventors"":[{""name"":""N. PETERS"",""sex"":0,""occ...",1096,187,1283,0.000277


In [32]:
import json
row_num = 20
type(sub_patents.iloc[row_num]['parsed_desc']), json.loads(sub_patents.iloc[row_num]['parsed_desc']), sub_patents.iloc[row_num].description

(str,
 {'inventors': [{'name': 'Michael Spelman',
    'sex': 0,
    'occupation': 'Mechanical engineers nec',
    'industry': 'Manufacturing machinery and equipment',
    'address': {'street': '',
     'city': 'Shreveport',
     'county': 'Caddo',
     'metro': '',
     'state': 'Louisiana',
     'region': '',
     'foreign': False,
     'full_address': 'Shreveport, Caddo, Louisiana'}}]},
 'Application filed January 7,1886. Serial No. 187,932. (No model.) To all zvhom it may concern: Be it known that I, Michael Spelman, of Shreveport, in the parish of Caddo and State of Louisiana, have invented a new and useful 5 Improvement in Car-Couplings, of which the following is a description. My present invention is an improvement upon the automatic coupling for which Letters Patent No. 330,166 were granted to me to November 10, 1885. The novel features constituting theimprovement are hereinafter fully described, and specifically indicated in the claims. In the accompanying drawings, Figur')

In [33]:
sub_patents.to_pickle(f'../int_data/sub_patents_4omini_{prompt_flag}_extrac_first_{TRUNCATING_AT}_chars_{sample_size}_docs_needs_exp.pkl')

In [41]:
sub_patents = pd.read_pickle(f'../int_data/sub_patents_4omini_{prompt_flag}_extrac_first_{TRUNCATING_AT}_chars_{sample_size}_docs_needs_exp.pkl')
# exploded_cols = pd.json_normalize(sub_patents.parsed_desc.apply(json.loads))
# sub_patents = pd.concat([sub_patents, exploded_cols], axis = 1)
# sub_patents

In [37]:
sub_patents.parsed_desc.isna().sum(), sub_patents.cost.sum()

(0, 63.03577934999997)

In [42]:
import json

def parse_json(x):
    try:
        return json.loads(x)
    except json.JSONDecodeError:
        return None  # or you can return {} or any default value

sub_patents['parsed_desc_parsed'] = sub_patents['parsed_desc'].apply(parse_json)
sub_patents['parsed_desc_type'] = sub_patents['parsed_desc_parsed'].apply(type)

# sub_patents.parsed_desc_type = sub_patents.parsed_desc.apply(type)
sub_patents.parsed_desc_type.value_counts()

parsed_desc_type
<class 'dict'>        308994
<class 'NoneType'>        41
Name: count, dtype: int64

In [43]:
# extracting fields
import json
def extract_inventors(text):
    parsed = json.loads(text)
    if parsed is not None and isinstance(parsed, dict):
        return parsed['inventors']
    else:
        return None
    
def extract_fields_afer_exploding(item, field):
    if not isinstance(item, dict):
        try:
            item = json.loads(item)
        except:
            return None
    # assert isinstance(item, dict), f"Not a dict after json.loads(): {item}"
    if not isinstance(item, dict):
        print(f"Not a dict after json.loads(): {item}")
    return item.get(field, None)

# def extract_fields_after_exploding(item, field):
#     # Example logic: extract the specific field from the inventor dictionary
#     return item.get(field, None) if isinstance(item, dict) else None

sub_patents['inventors'] = parallel_apply(sub_patents['parsed_desc'], extract_fields_afer_exploding,  field = 'inventors')

# exlpode by inventor
sub_patents_exploded = sub_patents.explode("inventors").reset_index(drop = True)

# extract from the explpoded
sub_patents_exploded['name'] = parallel_apply(sub_patents_exploded['inventors'], extract_fields_afer_exploding, field = 'name')
sub_patents_exploded['sex'] = parallel_apply(sub_patents_exploded['inventors'], extract_fields_afer_exploding, field = 'sex')
sub_patents_exploded['occupation'] = parallel_apply(sub_patents_exploded['inventors'], extract_fields_afer_exploding, field = 'occupation')
sub_patents_exploded['industry'] = parallel_apply(sub_patents_exploded['inventors'], extract_fields_afer_exploding, field = 'industry')
sub_patents_exploded['address'] = parallel_apply(sub_patents_exploded['inventors'], extract_fields_afer_exploding, field = 'address')
sub_patents_exploded['street'] = parallel_apply(sub_patents_exploded['address'], extract_fields_afer_exploding, field = 'street')
sub_patents_exploded['city'] = parallel_apply(sub_patents_exploded['address'], extract_fields_afer_exploding, field = 'city')
sub_patents_exploded['county'] = parallel_apply(sub_patents_exploded['address'], extract_fields_afer_exploding, field = 'county')
sub_patents_exploded['state'] = parallel_apply(sub_patents_exploded['address'], extract_fields_afer_exploding, field = 'state')
sub_patents_exploded['region'] = parallel_apply(sub_patents_exploded['address'], extract_fields_afer_exploding, field = 'region')
sub_patents_exploded['foreign'] = parallel_apply(sub_patents_exploded['address'], extract_fields_afer_exploding, field = 'foreign')
sub_patents_exploded['full_address'] = parallel_apply(sub_patents_exploded['address'], extract_fields_afer_exploding, field = 'full_address')

print(sub_patents_exploded.shape[0], sub_patents_exploded.description.nunique())

sub_patents_exploded

100%|██████████| 368026/368026 [00:01<00:00, 226803.45it/s]
2024-10-21 23:15:41,584 - distributed.utils_perf - WARNING - full garbage collections took 34% CPU time recently (threshold: 10%)
2024-10-21 23:15:44,735 - distributed.utils_perf - WARNING - full garbage collections took 43% CPU time recently (threshold: 10%)
100%|██████████| 368026/368026 [00:01<00:00, 228602.59it/s]
2024-10-21 23:15:49,075 - distributed.utils_perf - WARNING - full garbage collections took 43% CPU time recently (threshold: 10%)
2024-10-21 23:15:51,372 - distributed.utils_perf - WARNING - full garbage collections took 44% CPU time recently (threshold: 10%)
2024-10-21 23:15:54,078 - distributed.utils_perf - WARNING - full garbage collections took 44% CPU time recently (threshold: 10%)
2024-10-21 23:15:57,212 - distributed.utils_perf - WARNING - full garbage collections took 45% CPU time recently (threshold: 10%)
100%|██████████| 368026/368026 [00:01<00:00, 228051.41it/s]
2024-10-21 23:16:01,536 - distributed.ut

368026 308981


,publicationnumber,X,Publication.date,Title,description,year,parsed_desc,prompt_tokens,completion_tokens,total_tokens,...,occupation,industry,address,street,city,county,state,region,foreign,full_address
0,US343063A,47890,01/06/1886,None,"Application filed November 24, 1885. Serial No...",1886,"{""inventors"":[{""name"":""Charles F. Plumb"",""sex""...",1037,67,1104,...,Agricultural engineers,Agriculture and forestry,"{'street': '', 'city': 'Waterford', 'county': ...",,Waterford,Oakland,Michigan,,False,"Waterford, Oakland, Michigan"
1,US343064A,47891,01/06/1886,None,"Application filed April 1,1886. Serial No. 197...",1886,"{""inventors"":[{""name"":""William F. Price"",""sex""...",1035,73,1108,...,Engineering technicians nec,Agriculture and related industries,"{'street': '', 'city': 'West Caln township', '...",,West Caln township,Chester,Pennsylvania,,False,"West Caln township, Chester County, Pennsylvania"
2,US343065A,47892,01/06/1886,None,"Application filed April 16, 1886. Serial No. 1...",1886,"{""inventors"":[{""name"":""Thomas B. Purves"",""sex""...",1033,78,1111,...,Mechanical engineers,Mechanical engineering and industrial machinery,"{'street': '', 'city': 'Greenbush', 'county': ...",,Greenbush,Rensselaer,New York,,False,"Greenbush, Rensselaer, New York"
3,US343066A,47893,01/06/1886,None,"Application filed February 23, 1886. Serial No...",1886,"{""inventors"":[{""name"":""William D. Ready"",""sex""...",1022,65,1087,...,Engineering technicians nec,Manufacturing,"{'street': '', 'city': 'South Brooklyn', 'coun...",,South Brooklyn,Kings,New York,,False,"South Brooklyn, Kings, New York"
4,US343067A,47894,01/06/1886,None,"Application filed September 4, 1885. Serial No...",1886,"{""inventors"":[{""name"":""John F. Boeking"",""sex"":...",1036,61,1097,...,Engineers,Manufacturing,"{'street': '', 'city': 'Jackson', 'county': 'J...",,Jackson,Jackson,Michigan,,False,"Jackson, Jackson, Michigan"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368021,US215191A,50407,06/05/1879,None,"N. PETERS. PHOTO-LITHOGRAPHER. WASHINGTON, 0 C...",1879,"{""inventors"":[{""name"":""N. PETERS"",""sex"":0,""occ...",1096,187,1283,...,Photo-lithographer,Publishing industries (including software),"{'street': '', 'city': 'Washington', 'county':...",,Washington,,District of Columbia,,False,"Washington, D C"
368022,US215191A,50407,06/05/1879,None,"N. PETERS. PHOTO-LITHOGRAPHER. WASHINGTON, 0 C...",1879,"{""inventors"":[{""name"":""N. PETERS"",""sex"":0,""occ...",1096,187,1283,...,Graphic arts technicians,Manufacturing industries,"{'street': '', 'city': 'East Providence', 'cou...",,East Providence,,Rhode Island,,False,"East Providence, Rhode Island"
368023,US215191A,50407,06/05/1879,None,"N. PETERS. PHOTO-LITHOGRAPHER. WASHINGTON, 0 C...",1879,"{""inventors"":[{""name"":""N. PETERS"",""sex"":0,""occ...",1096,187,1283,...,Graphic arts technicians,Manufacturing industries,"{'street': '', 'city': 'Providence', 'county':...",,Providence,,Rhode Island,,False,"Providence, Rhode Island"
368024,US215192A,50408,06/05/1879,None,Sheets—Sheet 1. G. F. WILSON. Apparatus for Dr...,1879,"{""inventors"":[{""name"":""George F. Wilson"",""sex""...",1085,121,1206,...,Machinery engineers,Manufacturing machinery,"{'street': '', 'city': 'East Providence', 'cou...",,East Providence,Providence,Rhode Island,,False,"East Providence, Rhode Island"


In [46]:
# examine non-inventor docs
print(sub_patents['inventors'].isna().sum()) # 0

# examine inventors after explosion is null
print(sub_patents_exploded.inventors.isna().sum()) # 1/3000

58
116


### Post-processing

In [47]:
import numpy as np
sub_patents_exploded['street'] = sub_patents_exploded['street'].apply(lambda x:None if (x == np.nan or x == 'null' or x == '') else x)
sub_patents_exploded['city'] = sub_patents_exploded['city'].apply(lambda x:None if (x == np.nan or x == 'null' or x == '') else x)
sub_patents_exploded['county'] = sub_patents_exploded['county'].apply(lambda x:None if (x == np.nan or x == 'null' or x == '') else x)
sub_patents_exploded['state'] = sub_patents_exploded['state'].apply(lambda x:None if (x == np.nan or x == 'null' or x == '') else x)
sub_patents_exploded['region'] = sub_patents_exploded['region'].apply(lambda x:None if (x == np.nan or x == 'null' or x == '') else x)
sub_patents_exploded['fsull_address'] = sub_patents_exploded['full_address'].apply(lambda x:None if (x == np.nan or x == 'null' or x == '') else x)

In [ ]:
# sub_patents_exploded[~sub_patents_exploded.inventorsnames1.isna()][['description', 'name', 'pname','sname']] # inventorsnames1 is from Filippo's code, whose extraction is not supplementary to the results here

In [71]:
# unify foreign
mapping = {
    'False': False,
    'false': False,
    'False ': False,
    'True': True,
    'true': True,
    'True ': True,
    'null': None,
    'NULL': None,
    '': None
}

# Apply the mapping
sub_patents_exploded.foreign = sub_patents_exploded.foreign.apply(lambda x:mapping[x] if x in mapping.keys() else x)

# Display the unified result
sub_patents_exploded.foreign.value_counts(), sub_patents_exploded.foreign.isna().sum()

(foreign
 False    306285
 True      19413
 Name: count, dtype: int64,
 42328)

In [77]:
sex_mapping = {
    0: 0,
    1: 1,
    'null': None,
    'NULL': None,
    '': None,
    '1':1,
    '0':0
}

# Apply the mapping
sub_patents_exploded.sex = sub_patents_exploded.sex.apply(lambda x:sex_mapping[x] if x in sex_mapping.keys() else x)

# Display the unified result
sub_patents_exploded.sex.value_counts(), sub_patents_exploded.sex.isna().sum()

(sex
 0.0    348814
 1.0      3994
 Name: count, dtype: int64,
 15218)

### State

In [66]:
sub_patents_exploded.state.value_counts(), sub_patents_exploded.state.nunique()

(state
 New York                     59201
 Pennsylvania                 31870
 Massachusetts                28543
 Illinois                     24257
 Ohio                         21994
                              ...  
 Ohto                             1
 United States of Colombia        1
 Algeria                          1
 Puebla                           1
 Solothurn                        1
 Name: count, Length: 296, dtype: int64,
 296)

In [53]:
import numpy as np
state_corrections = {
    'D.C.': 'District of Columbia',
    'D. C.': 'District of Columbia',
    'D C': 'District of Columbia',
    'DC': 'District of Columbia',
    'Washington, D. C.':'District of Columbia',
    'NY': 'New York',
    'N. Y.': 'New York',
    'NEW YORK': 'New York',
    'PA': 'Pennsylvania',
    'INDIANA': 'Indiana',
    'Washington Territory': 'Washington',
    'Dakota': 'North Dakota',  # Assuming North Dakota since it's unclear
    'Dakota Territory': 'North Dakota',  # Similar assumption as Dakota
    'Colorado Territory': 'Colorado',
    'Indian Territory': 'Oklahoma',  # Modern-day Oklahoma
    'Montana Territory': 'Montana',
    'Utah Territory': 'Utah',
    'N/A':np.nan,
    'null':np.nan,
    'NA': np.nan,
    '':np.nan
}

# Apply the corrections
sub_patents_exploded['state'] = sub_patents_exploded['state'].replace(state_corrections)


In [54]:
sub_patents_exploded.state.value_counts().to_dict(), sub_patents_exploded.state.nunique() 
# There are other countries here such as Denmark, we don't have to clean them.
# These would be automatically filterd out in the comparison (blocked using state)

({'New York': 59201,
  'Pennsylvania': 31870,
  'Massachusetts': 28543,
  'Illinois': 24257,
  'Ohio': 21994,
  'District of Columbia': 16427,
  'Connecticut': 12681,
  'New Jersey': 12392,
  'Michigan': 10520,
  'Indiana': 8899,
  'Missouri': 8735,
  'California': 7889,
  'Iowa': 6668,
  'Wisconsin': 6276,
  'Rhode Island': 3990,
  'Maryland': 3943,
  'Kentucky': 3234,
  'Texas': 3158,
  'Minnesota': 3148,
  'Maine': 2881,
  'Kansas': 2842,
  'New Hampshire': 2166,
  'Tennessee': 2081,
  'Virginia': 1941,
  'Georgia': 1859,
  'Vermont': 1788,
  'Nebraska': 1438,
  'Louisiana': 1362,
  'England': 1339,
  'Colorado': 1321,
  'Ontario': 1232,
  'North Carolina': 1057,
  'Alabama': 1037,
  'West Virginia': 1019,
  'Mississippi': 932,
  'South Carolina': 813,
  'Oregon': 762,
  'Arkansas': 750,
  'Delaware': 730,
  'Quebec': 561,
  'Germany': 499,
  'North Dakota': 452,
  'Prussia': 451,
  'Florida': 415,
  'France': 345,
  'Washington': 324,
  'Nevada': 321,
  'Utah': 292,
  'Montana': 22

### names

In [67]:
# make capital
sub_patents_exploded.street = sub_patents_exploded.street.apply(lambda x:x.upper() if x else x)
sub_patents_exploded['name'] =sub_patents_exploded.name.apply(lambda x:x.upper() if x else x)
sub_patents_exploded['pname']=sub_patents_exploded.name.apply(lambda x:' '.join(x.split(' ')[:-1]) if x else x)
sub_patents_exploded['sname'] =sub_patents_exploded.name.apply(lambda x:x.split(' ')[-1] if x else x)

# drop name where it is UNKNOWN
sub_patents_exploded['name'] =sub_patents_exploded.name.apply(lambda x:None if (x == 'UNKNOWN' or x == '' or x == 'NULL' or x == 'NA' or x == 'NAN') else x)
sub_patents_exploded['pname'] =sub_patents_exploded.pname.apply(lambda x:None if (x == 'UNKNOWN' or x == '' or x == 'NULL' or x == 'NA' or x == 'NAN') else x)
sub_patents_exploded['sname'] =sub_patents_exploded.sname.apply(lambda x:None if (x == 'UNKNOWN' or x == '' or x == 'NULL' or x == 'NA' or x == 'NAN') else x)

# create unique indentifier for each inventor
sub_patents_exploded['inventor_no'] = sub_patents_exploded.groupby('publicationnumber').cumcount()

In [72]:
sub_patents_exploded

,publicationnumber,X,Publication.date,Title,description,year,parsed_desc,prompt_tokens,completion_tokens,total_tokens,...,city,county,state,region,foreign,full_address,fsull_address,pname,sname,inventor_no
0,US343063A,47890,01/06/1886,None,"Application filed November 24, 1885. Serial No...",1886,"{""inventors"":[{""name"":""Charles F. Plumb"",""sex""...",1037,67,1104,...,Waterford,Oakland,Michigan,None,False,"Waterford, Oakland, Michigan","Waterford, Oakland, Michigan",CHARLES F.,PLUMB,0
1,US343064A,47891,01/06/1886,None,"Application filed April 1,1886. Serial No. 197...",1886,"{""inventors"":[{""name"":""William F. Price"",""sex""...",1035,73,1108,...,West Caln township,Chester,Pennsylvania,None,False,"West Caln township, Chester County, Pennsylvania","West Caln township, Chester County, Pennsylvania",WILLIAM F.,PRICE,0
2,US343065A,47892,01/06/1886,None,"Application filed April 16, 1886. Serial No. 1...",1886,"{""inventors"":[{""name"":""Thomas B. Purves"",""sex""...",1033,78,1111,...,Greenbush,Rensselaer,New York,None,False,"Greenbush, Rensselaer, New York","Greenbush, Rensselaer, New York",THOMAS B.,PURVES,0
3,US343066A,47893,01/06/1886,None,"Application filed February 23, 1886. Serial No...",1886,"{""inventors"":[{""name"":""William D. Ready"",""sex""...",1022,65,1087,...,South Brooklyn,Kings,New York,None,False,"South Brooklyn, Kings, New York","South Brooklyn, Kings, New York",WILLIAM D.,READY,0
4,US343067A,47894,01/06/1886,None,"Application filed September 4, 1885. Serial No...",1886,"{""inventors"":[{""name"":""John F. Boeking"",""sex"":...",1036,61,1097,...,Jackson,Jackson,Michigan,None,False,"Jackson, Jackson, Michigan","Jackson, Jackson, Michigan",JOHN F.,BOEKING,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368021,US215191A,50407,06/05/1879,None,"N. PETERS. PHOTO-LITHOGRAPHER. WASHINGTON, 0 C...",1879,"{""inventors"":[{""name"":""N. PETERS"",""sex"":0,""occ...",1096,187,1283,...,Washington,None,District of Columbia,None,False,"Washington, D C","Washington, D C",N.,PETERS,0
368022,US215191A,50407,06/05/1879,None,"N. PETERS. PHOTO-LITHOGRAPHER. WASHINGTON, 0 C...",1879,"{""inventors"":[{""name"":""N. PETERS"",""sex"":0,""occ...",1096,187,1283,...,East Providence,None,Rhode Island,None,False,"East Providence, Rhode Island","East Providence, Rhode Island",GEORGE F.,WILSON,1
368023,US215191A,50407,06/05/1879,None,"N. PETERS. PHOTO-LITHOGRAPHER. WASHINGTON, 0 C...",1879,"{""inventors"":[{""name"":""N. PETERS"",""sex"":0,""occ...",1096,187,1283,...,Providence,None,Rhode Island,None,False,"Providence, Rhode Island","Providence, Rhode Island",CHARLES A.,CATLIN,2
368024,US215192A,50408,06/05/1879,None,Sheets—Sheet 1. G. F. WILSON. Apparatus for Dr...,1879,"{""inventors"":[{""name"":""George F. Wilson"",""sex""...",1085,121,1206,...,East Providence,Providence,Rhode Island,None,False,"East Providence, Rhode Island","East Providence, Rhode Island",GEORGE F.,WILSON,0


In [69]:
sub_patents_exploded.iloc[51].description

'Application filed September 8, 1884. Serial No. 142,487. (No model.) To all lulwm it may concern: Be it known that we, George Crompton and Horace Wyman, of the city and county of Worcester, Massachusetts, have invented 5 an Improvement in Looms for Weaving Tufted Fabrics, of which the following description, in connection with the accompanying drawings, is a specification, like letters on the drawings representing like parts. i o This invention relates to looms of the class adapted for the production of tufted fabrics— such, for instance, as moquette carpet—the object of the invention being to '

In [74]:
sub_patents_exploded.isna().sum()

publicationnumber          0
X                          0
Publication.date           0
Title                 367736
description                0
year                       0
parsed_desc                0
prompt_tokens              0
completion_tokens          0
total_tokens               0
cost                       0
parsed_desc_parsed        41
parsed_desc_type           0
inventors                116
name                    8144
sex                    14693
occupation              3793
industry                3794
address                40993
street                350566
city                   47957
county                111715
state                  58258
region                357296
foreign                42328
full_address           41034
fsull_address          54870
pname                  10044
sname                   8160
inventor_no                0
dtype: int64

In [78]:
# check sex and name
sub_patents_exploded.name.value_counts(),sub_patents_exploded.sname.value_counts(), sub_patents_exploded.pname.value_counts(), sub_patents_exploded.sex.value_counts(), 

(name
 N. PETERS            12182
 THOMAS A. EDISON       540
 PETERS                 501
 H. PETERS              351
 ELIHU THOMSON          201
                      ...  
 MARY D. BRINE            1
 ZEBINA L. BRAGDON        1
 ALLEN BAILEY             1
 JOHN G. BOYD             1
 H. J. DURGIN             1
 Name: count, Length: 181066, dtype: int64,
 sname
 PETERS      14044
 SMITH        4186
 JR.          4117
 BROWN        1886
 JOHNSON      1689
             ...  
 HEGI            1
 BINKERT         1
 KINSTLER        1
 RABENAU         1
 BIBON           1
 Name: count, Length: 44635, dtype: int64,
 pname
 N.              12535
 JOHN             9073
 WILLIAM          7473
 JAMES            4374
 GEORGE           3894
                 ...  
 M. E. B.            1
 PARDON C.           1
 ALBERTH             1
 VINCENT ABEL        1
 R. Η. H.            1
 Name: count, Length: 40725, dtype: int64,
 sex
 0.0    348814
 1.0      3994
 Name: count, dtype: int64)

In [79]:
# sub_patents_exploded.to_pickle(f'../int_data/sub_patents_4o_extrac_fist{TRUNCATING_AT}chars_10_docs.pkl')
# sub_patents_exploded.to_pickle(f'../int_data/sub_patents_4omini_extrac_fist_full_chars_1000_docs.pkl')

sub_patents_exploded.to_pickle(f'../int_data/sub_patents_4omini_{prompt_flag}_extrac_first_{TRUNCATING_AT}_chars_{sample_size}_docs.pkl')

In [19]:
import pandas as pd

prompt_flag = 'ver0_2'
TRUNCATING_AT=600
# sample_size = 3000
sample_size = 309035
# sub_patents_4o_extrac_600chars_3000 = pd.read_pickle(f'../int_data/sub_patents_4omini_extrac_fist_full_chars_{sample_size}_docs.pkl')
sub_patents_4o_extrac_600chars_3000 = pd.read_pickle(f'../int_data/sub_patents_4omini_{prompt_flag}_extrac_first_{TRUNCATING_AT}_chars_{sample_size}_docs.pkl')
sub_patents_4o_extrac_600chars_3000

,publicationnumber,X,Publication.date,Title,description,year,parsed_desc,prompt_tokens,completion_tokens,total_tokens,...,city,county,state,region,foreign,full_address,fsull_address,pname,sname,inventor_no
0,US343063A,47890,01/06/1886,None,"Application filed November 24, 1885. Serial No...",1886,"{""inventors"":[{""name"":""Charles F. Plumb"",""sex""...",1037,67,1104,...,Waterford,Oakland,Michigan,None,False,"Waterford, Oakland, Michigan","Waterford, Oakland, Michigan",CHARLES F.,PLUMB,0
1,US343064A,47891,01/06/1886,None,"Application filed April 1,1886. Serial No. 197...",1886,"{""inventors"":[{""name"":""William F. Price"",""sex""...",1035,73,1108,...,West Caln township,Chester,Pennsylvania,None,False,"West Caln township, Chester County, Pennsylvania","West Caln township, Chester County, Pennsylvania",WILLIAM F.,PRICE,0
2,US343065A,47892,01/06/1886,None,"Application filed April 16, 1886. Serial No. 1...",1886,"{""inventors"":[{""name"":""Thomas B. Purves"",""sex""...",1033,78,1111,...,Greenbush,Rensselaer,New York,None,False,"Greenbush, Rensselaer, New York","Greenbush, Rensselaer, New York",THOMAS B.,PURVES,0
3,US343066A,47893,01/06/1886,None,"Application filed February 23, 1886. Serial No...",1886,"{""inventors"":[{""name"":""William D. Ready"",""sex""...",1022,65,1087,...,South Brooklyn,Kings,New York,None,False,"South Brooklyn, Kings, New York","South Brooklyn, Kings, New York",WILLIAM D.,READY,0
4,US343067A,47894,01/06/1886,None,"Application filed September 4, 1885. Serial No...",1886,"{""inventors"":[{""name"":""John F. Boeking"",""sex"":...",1036,61,1097,...,Jackson,Jackson,Michigan,None,False,"Jackson, Jackson, Michigan","Jackson, Jackson, Michigan",JOHN F.,BOEKING,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368021,US215191A,50407,06/05/1879,None,"N. PETERS. PHOTO-LITHOGRAPHER. WASHINGTON, 0 C...",1879,"{""inventors"":[{""name"":""N. PETERS"",""sex"":0,""occ...",1096,187,1283,...,Washington,None,District of Columbia,None,False,"Washington, D C","Washington, D C",N.,PETERS,0
368022,US215191A,50407,06/05/1879,None,"N. PETERS. PHOTO-LITHOGRAPHER. WASHINGTON, 0 C...",1879,"{""inventors"":[{""name"":""N. PETERS"",""sex"":0,""occ...",1096,187,1283,...,East Providence,None,Rhode Island,None,False,"East Providence, Rhode Island","East Providence, Rhode Island",GEORGE F.,WILSON,1
368023,US215191A,50407,06/05/1879,None,"N. PETERS. PHOTO-LITHOGRAPHER. WASHINGTON, 0 C...",1879,"{""inventors"":[{""name"":""N. PETERS"",""sex"":0,""occ...",1096,187,1283,...,Providence,None,Rhode Island,None,False,"Providence, Rhode Island","Providence, Rhode Island",CHARLES A.,CATLIN,2
368024,US215192A,50408,06/05/1879,None,Sheets—Sheet 1. G. F. WILSON. Apparatus for Dr...,1879,"{""inventors"":[{""name"":""George F. Wilson"",""sex""...",1085,121,1206,...,East Providence,Providence,Rhode Island,None,False,"East Providence, Rhode Island","East Providence, Rhode Island",GEORGE F.,WILSON,0


In [20]:
sub_patents_4o_extrac_600chars_3000.street.value_counts()

street
39 & 41 PARK PLACE    817
PARK PLACE            315
5                     177
PHILADELPHIA          138
CHICAGO               132
                     ... 
LANARK                  1
26 LEVER STREET         1
GRANGE FARM             1
SWATARA TOWNSHIP        1
GAFFNEY CITY            1
Name: count, Length: 8938, dtype: int64

In [21]:
sub_patents_4o_extrac_600chars_3000.occupation.value_counts()

occupation
Engineering technicians nec                         148395
Industrial machinery or tools engineers              34676
Mechanical engineers                                 25468
Engineers                                            10542
Agricultural engineers                                9900
                                                     ...  
Engineers in Electric and Electronic Engineering         1
Engineers concerned with industrial machinery            1
Engineers in farming or agricultural machinery           1
Inventors in general mechanics                           1
Legal professionals nec                                  1
Name: count, Length: 23038, dtype: int64

In [22]:
sub_patents_4o_extrac_600chars_3000.industry.value_counts()

industry
Manufacturing                                             88936
Machinery and equipment manufacturing                     10803
Manufacturing machinery and equipment                      8751
Manufacturing machinery                                    8122
Aircraft and parts                                         6708
                                                          ...  
Manufacturing and processing of cotton products               1
Manufacturing and processing of furniture and fixtures        1
Manufacturing or repair of textile machinery                  1
Manufacturing (arts and related services)                     1
Manufacturing of Lamp-Black                                   1
Name: count, Length: 25901, dtype: int64

In [23]:
sub_patents_4o_extrac_600chars_3000.description.apply(len)

0         600
1         600
2         600
3         600
4         600
         ... 
368021    600
368022    600
368023    600
368024    600
368025    600
Name: description, Length: 368026, dtype: int64

In [24]:
sub_patents_4o_extrac_600chars_3000.isna().sum()

publicationnumber          0
X                          0
Publication.date           0
Title                 367736
description                0
year                       0
parsed_desc                0
prompt_tokens              0
completion_tokens          0
total_tokens               0
cost                       0
parsed_desc_parsed        41
parsed_desc_type           0
inventors                116
name                    8144
sex                    15218
occupation              3793
industry                3794
address                40993
street                350566
city                   47957
county                111715
state                  58258
region                357296
foreign                42328
full_address           41034
fsull_address          54870
pname                  10044
sname                   8160
inventor_no                0
dtype: int64

### occupation

In [25]:
try:
    from histocc import OccCANINE # HISCO prediciton class
except:
    !rm -rf OccCANINE # Remove existing
    !git clone https://github.com/christianvedels/OccCANINE
    !pip install OccCANINE/
    !pip install unidecode
    from histocc import OccCANINE # HISCO prediciton class
model = OccCANINE() # This downloads and initializes the model from HuggingFace

/users/xiet13/.local/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [26]:
# check hisco numbers
len(model.key.values()), len(model.key_desc.values())

(1919, 1919)

In [27]:
hisco_table = {k:v for (k,v) in zip(model.key.values(), model.key_desc.values())}
len(hisco_table)

1919

In [28]:
sample = sub_patents_4o_extrac_600chars_3000.sample(frac = 0.001, random_state = 42)['occupation'].iloc[0:3].tolist()
print(sample)
model.predict(
    # ["tailor of the finest suits", "local boiler maker", "train's fireman"],
    sample,
    get_dict = True # Simple output
)

['Industrial machinery or tools engineers', 'Engineering technicians nec', 'Engineering technicians nec']


[[[2420, 0.9860619, 'Industrial Machinery and Tools Engineer']],
 [[1400, 0.9316264, 'Physical Science Technician, Specialisation Unknown']],
 [[1400, 0.9316264, 'Physical Science Technician, Specialisation Unknown']]]

In [29]:
sample = sub_patents_4o_extrac_600chars_3000.sample(frac = 0.001, random_state = 42)['description'].iloc[0:3].tolist()
print(sample)
model.predict(
    # ["tailor of the finest suits", "local boiler maker", "train's fireman"],
    sample,
    get_dict = True # Simple output
)

['Application filed October 22,1831. (No model.) To all whom it may concern; Be it known that I, Frederick B. Miles, a citizen of the United States, residing in Philadelphia, Pennsylvania, haveinvented cer5 tain Improvements in Taper Attachments for Lathes, of which the following is a specification. My invention consists of a device to be attached to a lathe for so controlling one of the to slides of the slide-rest that the cutting-tool, instead of taking a direct course parallel with the axes of the head and tail stocks, will move transversely as the slide-rest is traversed, thereby imparting a', 'Application filed June 2-2, 1886. Serial No. 205,872. (No model.) To all whom it may concern: Be it known that I, Johannes Grube, of the city of New York, in the county and State of New York, have invented certain new and 5 useful Improvements in Extension-Tables, of which the following is a specification. This invention-relates to an extension-table that is capable of extension in four dire

[[], [], []]

In [30]:
model.predict(
    # ["tailor of the finest suits", "local boiler maker", "train's fireman"],
    sample,
    lang = 'en',
    get_dict = True # Simple output
)

[[], [], []]

In [ ]:
# Fast performance for many observations
x_en = model.predict(
    sub_patents_4o_extrac_600chars_3000['occupation'].fillna(''), # Need ad-hoc fillna
    lang = "en",
    threshold = 0.22 # Optimal for f1
    )

x = model.predict(
    sub_patents_4o_extrac_600chars_3000['occupation'].fillna(''), # Need ad-hoc fillna
    threshold = 0.22 # Optimal for f1
    )

In [ ]:
x.isna().sum(), x_en.isna().sum()

In [ ]:
import numpy as np

sub_patents_4o_extrac_600chars_3000 = pd.concat([sub_patents_4o_extrac_600chars_3000, x_en[['hisco_1', 'prob_1', 'desc_1']].rename(columns = {'hisco_1':'hisco_en', 'prob_1':'prob_en', 'desc_1':'desc_en'})], axis = 1) # concat x_en first

# NAs, around 10%
sub_patents_4o_extrac_600chars_3000.desc_en = sub_patents_4o_extrac_600chars_3000.desc_en.apply(lambda x: np.nan if x.strip() == 'No pred' else x)  
sub_patents_4o_extrac_600chars_3000.hisco_en = sub_patents_4o_extrac_600chars_3000.hisco_en.apply(lambda x: np.nan if x.strip() == 'nan' else x)

# Then we concat prediction result where language is not specified
sub_patents_4o_extrac_600chars_3000 = pd.concat([sub_patents_4o_extrac_600chars_3000, x[['hisco_1', 'prob_1', 'desc_1']].rename(columns = {'hisco_1':'hisco_nolang', 'prob_1':'prob_nolang', 'desc_1':'desc_nolang'})], axis = 1) # concat x
# NAs
sub_patents_4o_extrac_600chars_3000.desc_nolang = sub_patents_4o_extrac_600chars_3000.desc_nolang.apply(lambda x: np.nan if x.strip() == 'No pred' else x)  
sub_patents_4o_extrac_600chars_3000.hisco_nolang = sub_patents_4o_extrac_600chars_3000.hisco_nolang.apply(lambda x: np.nan if x.strip() == 'nan' else x)

In [ ]:
sub_patents_4o_extrac_600chars_3000.hisco_en.isna().sum(), sub_patents_4o_extrac_600chars_3000.prob_en.isna().sum(), sub_patents_4o_extrac_600chars_3000.desc_en.isna().sum()

In [ ]:
sub_patents_4o_extrac_600chars_3000.hisco_nolang.isna().sum(), sub_patents_4o_extrac_600chars_3000.prob_nolang.isna().sum(), sub_patents_4o_extrac_600chars_3000.desc_nolang.isna().sum()

In [ ]:
sub_patents_4o_extrac_600chars_3000.hisco_nolang.value_counts(), sub_patents_4o_extrac_600chars_3000.prob_nolang.value_counts(), sub_patents_4o_extrac_600chars_3000.desc_nolang.value_counts()

In [ ]:
sub_patents_4o_extrac_600chars_3000['hisco'] = sub_patents_4o_extrac_600chars_3000.apply(lambda x: x.hisco_en if not pd.isna(x.hisco_en) else x.hisco_nolang, axis = 1)
sub_patents_4o_extrac_600chars_3000['prob'] = sub_patents_4o_extrac_600chars_3000.apply(lambda x: x.prob_en if (not pd.isna(x.hisco_en) or not pd.isna(x.desc_en))  else x.prob_nolang, axis = 1)
sub_patents_4o_extrac_600chars_3000['desc'] = sub_patents_4o_extrac_600chars_3000.apply(lambda x: x.desc_en if not pd.isna(x.desc_en) else x.desc_nolang, axis = 1)

In [ ]:
sub_patents_4o_extrac_600chars_3000.hisco.isna().sum(), sub_patents_4o_extrac_600chars_3000.prob.isna().sum(), sub_patents_4o_extrac_600chars_3000.desc.isna().sum()

In [ ]:
sub_patents_4o_extrac_600chars_3000.drop(columns = ['hisco_en',	'prob_en',	'desc_en',	'hisco_nolang',	'prob_nolang',	'desc_nolang']).to_pickle('../int_data/patent_3000_sample_ver_0_2_hisco_alignment.pkl')
# sub_patents_4o_extrac_600chars_3000.to_pickle('../int_data/patent_hisco_alignment.pkl')
sub_patents_4o_extrac_600chars_3000.to_pickle('../int_data/all_patent_hisco_alignment.pkl')

In [ ]:
sub_patents_4o_extrac_600chars_3000

In [ ]:
blocker

## File Search (blocked by no access to Playground)
Workaround:
- TODO: use David's script to create/use an assistant
- Ask Filippo for OpenAI account
- Get another OpenAI account
- Wait for OpenAI to fix the bug in assistant API: response_format and tools can't be specified for one assistant

#### Build/use an assistant

In [ ]:
# response_format_occ_ind = {
#     "type": "json_schema",
#     "json_schema": {
#         "name": "occupation_industry_of_equipment",
#         "schema": {
#             "type": "object",
#             "properties": {
#                 "occupation": {
#                     "type": "string",
#                     "description": "The occupation which most closely matches the patent description and/or the invention given. This should come only from the markdown document you can access through your file search documents (where each row is a separate classification)."
#                 },
#                 "industry": {
#                     "type": "string",
#                     "description": "The industry which most closely matches the patent description and/or the invention given. This should come only from the markdown document you can access through your file search documents (where each row is a separate classification)."
#                 }
#             },
#             "required": ["occupation", "industry"],
#             "additionalProperties": False
#         }
#     }

# }

# # response_format_occ_ind = {
# #     "type": "object",
# #     "json_schema": {
# #         "name": "occupation_industry_of_equipment",
# #         "schema": {
# #             "type": "json_schema",
# #             "properties": {
# #                 "occupation": {
# #                     "type": "string",
# #                     "description": "The occupation which most closely matches the patent description and/or the invention given. This should come only from the markdown document you can access through your file search documents (where each row is a separate classification)."
# #                 },
# #                 "industry": {
# #                     "type": "string",
# #                     "description": "The industry which most closely matches the patent description and/or the invention given. This should come only from the markdown document you can access through your file search documents (where each row is a separate classification)."
# #                 }
# #             },
# #             "required": ["occupation", "industry"],
# #             "additionalProperties": False
# #         }
# #     }

# }

In [ ]:
# one-time run
try:
    from openai import OpenAI
except ImportError:
    !pip install openai
    from openai import OpenAI

from openai import OpenAI
 
# assistant_id = 'asst_8Zt26PUMDTJJOgXZXuW8zlgT' # family
client = OpenAI()

# find
# client.beta.assistants.list()


# md building
# import pandas as pd
# hisco_table = pd.DataFrame.from_dict(hisco_table, orient = 'index').reset_index()
# hisco_table.rename(columns = {'index':'code', 0:'description'}, inplace = True)
# def build_markdown(df, filename, headers):
#     markdown = f"# {filename.replace('_', ' ').title()}\n\n"
#     markdown += "| " + " | ".join(headers) + " |\n"
#     markdown += "| " + " | ".join(['---']*len(headers)) + " |\n"
#     for _, row in df.iterrows():
#         markdown += "| " + " | ".join(str(row[header]) for header in headers) + " |\n"
#     with open(f'../aux_data/{filename}.md', 'w') as f:
#         f.write(markdown)
#         print(f"done writing to ../aux_data/{filename}.md")

# build_markdown(hisco_table, 'hisco_table', ['code', 'description'])

# file_search assistant
# assistant = client.beta.assistants.create(
#   name="HISCO Occupation Classification",
#   instructions=f'''You are an expert in patent classification. Use the file search tool with the file you have been provided. The files are lists of occupation classifications, where each row is a seperate classification. You will be given a prompt of patent description, and you need to find the occupation classification from the document in the file search.  Always response with exactly the industry category
#       return code and description of the classification, for example, "'2410': 'Mechanical Engineer, General'".'''
#   # model="gpt-4o-2024-08-06",
#   model="gpt-4o-mini",
#   temperature=0.05,
#   # metadata=metadata,
#   tools=[{"type": "file_search"}],
#   # response_format=response_format_occ_ind
# )

# # point tool
# assistant = client.beta.assistants.update(
#   assistant_id=assistant.id,
#   tools=[{"type": "file_search"}],
#   # tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
# )



In [ ]:
# sub-sample again
sub_patents = sub_patents.sample(frac=0.05, random_state = 42)

In [ ]:
assistant_id = 'asst_6s7DPvmOSapQkXqvNpP01TWj'

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import pandas as pd
import multiprocessing as mp

cpu_count = mp.cpu_count()

# Define a function to process each row
def process_row(row):
    try:
        # Create a thread for the current row
        thread = client.beta.threads.create(
            messages=[{
                "role": "user",
                "content": f"Patent description: {row.description}"
            }]
        )
        
        # Create and poll the run
        run = client.beta.threads.runs.create_and_poll(
            thread_id=thread.id, 
            assistant_id=assistant_id
            # assistant_id=assistant.id
        )

        # run error handling
        max_retries = 3
        while run is None and max_retries >= 0:
            if max_retries == 0:
                raise ValueError(f"Run is None for row {row.name}, max retries reached.")
                return row
            print(f"Run is None for row {row.publicationnumber}, retrying...")
            max_retries -= 1
            # rerun
            run = client.beta.threads.runs.create_and_poll(
                thread_id=thread.id, 
                # assistant_id=assistant_id
            assistant_id=assistant.id
            )
            
        prompt_tokens, completion_tokens = run.usage.prompt_tokens, run.usage.completion_tokens, 
        row.prompt_tokens = prompt_tokens
        row.completion_tokens = completion_tokens

        # Retrieve messages from the thread
        messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))
        
        # Process the first message's content
        message_content = None
        if isinstance(messages, list) and len(messages) > 0:
            message = messages[0]
            try:
                message_content = message.content[0].text
            except:
                message_content = message.content.text
                
        if not message_content or message_content == "":
            print(messages)
            raise ValueError("No message content found.")
        
        annotations = message_content.annotations
        citations = []
        for index, annotation in enumerate(annotations):
            message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
            if file_citation := getattr(annotation, "file_citation", None):
                cited_file = client.files.retrieve(file_citation.file_id)
                citations.append(f"[{index}] {cited_file.filename}")

        # Return the processed data
        row['RAGed'] = message_content

    except Exception as e:
        # Handle any exceptions that occur during processing
        print(f"An error occurred for row {row.name}: {e}")
        row['RAGed'] = None
    return row

# List to hold the results
results = []

# Use ThreadPoolExecutor to run multiple threads in parallel
with ThreadPoolExecutor(max_workers=cpu_count-2) as executor:
    # Submit all row processing tasks to the executor
    futures = {executor.submit(process_row, row): row for _, row in sub_patents.iterrows()}
    
    # Use tqdm to show progress
    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing Rows"):
        # Append the result to the results list
        results.append(future.result())

# Convert the list of results back to a DataFrame
sub_patents = pd.DataFrame(results)


In [ ]:
import re

# sub_patents['hisco_code'] = sub_patents.RAGed.apply(lambda x:x.value.split(':')[0].strip().strip("'").strip("`").strip('[').strip(']').strip('"').strip("'"))
sub_patents['hisco_code'] = sub_patents.RAGed.apply(lambda x:x.value.split(':')[0].strip().strip("'`[]\""))
# sub_patents['hisco_desc'] = sub_patents.RAGed.apply(lambda x:x.value.split(':')[1].strip().strip("'").replace('[0]', '').strip("`").strip("】").strip("'"))
def clean_hisco_desc(value):
    # Split by colon and clean the second part for 'hisco_desc'
    desc = value.split(':')[1].strip().replace('[0]', '').strip("'`【】")
    # Remove unwanted backticks, newline, and additional patterns
    desc = re.sub(r'[`\n]|\d+$', '', desc).strip()
    return desc 

sub_patents['hisco_desc'] = sub_patents.RAGed.apply(lambda x:x.value.split(':')[1].strip().replace('[0]', '').strip("'`【】"))
sub_patents['hisco_desc'] = sub_patents.RAGed.apply(lambda x: clean_hisco_desc(x.value))

In [ ]:
sub_patents.to_pickle('../int_data/aligned_hisco_150_cleaned.pkl')

In [ ]:
# # read to check
# import pandas as pd
# sub_patents = pd.read_pickle('../int_data/aligned_hisco_150.pkl')
# sub_patents

## (deprecatd) pred with NER model

In [ ]:
sub_patents = patent.sample(100, random_state = 42)
sub_patents

In [ ]:
sub_patents.iloc[0].description	

In [ ]:
nlp = spacy.load("../model/name_addr_md")
doc = nlp(sub_patents.iloc[95].description)
from spacy import displacy
displacy.render(doc, style="ent")

In [ ]:
from collections import defaultdict
from tqdm import tqdm

def attach_ent_prob(text, POS_UP_BOUND = 300):
    if type(text) != list:
        text = list(text)
    docs = list(nlp.pipe(text))

    # Number of alternate analyses to consider. More is slower, and not necessarily better -- you need to experiment on your problem.
    beam_width = 16
    # This clips solutions at each step. We multiply the score of the top-ranked action by this value, and use the result as a threshold. This prevents the parser from exploring options that look very unlikely, saving a bit of efficiency. Accuracy may also improve, because we've trained on greedy objective.
    beam_density = 0.001
    beams = nlp.get_pipe('ner').beam_parse(docs, beam_width=beam_width, beam_density=beam_density)
    
    document_ent_scores = {}
    final = {}
    for doc, beam in zip(docs, beams):
        entity_scores = defaultdict(float)
        for score, ents in nlp.get_pipe('ner').moves.get_beam_parses(beam):
            for start, end, label in ents:
                if start <= POS_UP_BOUND:  # Consider entities within first 150 positions
                    entity_scores[(start, end, label)] += score
        document_ent_scores.update({doc:entity_scores})
    
    for i, (doc, entity_scores) in enumerate(document_ent_scores.items()):
        l= []
        for k, v in entity_scores.items():
            l.append({'start': k[0], 'end': k[1], 'label': k[2], 'prob' : v})
        sorted_entities = sorted(l, key= lambda x: (x['prob'], x['label']), reverse = True)
        
        # for entity in sorted_entities:
        #     print(entity, doc[entity['start']:entity['end']])
        # print(f"finished {doc.text}")
        
        # TO DO
        # sort by prob, descending
        # filter out anything appear after pos 150
        # Add restriction: GPE num equals to PERSON num
        # limit each to be within 4
        
        # Applying restrictions for GPE and PERSON
        gpe_count, person_count = 0, 0
        for dct in sorted_entities:
            start, end, label, score = list(dct.values())
            if label == 'GPE':
                gpe_count += 1
            elif label == 'PERSON':
                person_count += 1
        
        # Ensure GPE count equals PERSON count
        max_count = min(gpe_count, person_count, 3)  # Limit each to max 3
        
        # Filter and print entities
        final_entities = []
        for dct in sorted_entities:
            start, end, label, score = list(dct.values())
            str = doc[start: end].text
            if (label == 'GPE' or label == 'PERSON') and len([e for e in final_entities if e['label'] == label]) < max_count:
                final_entities.append({'start': start, 'end': end, 'label': label, 'prob': score, 'str':str})
        final.update({i: final_entities})
        # print(f"\nfinished: {doc.text}\n")
    
            
        
    assert len(document_ent_scores) == len(final)
    return final
entities = tqdm(attach_ent_prob(sub_patents.description.tolist(), POS_UP_BOUND = 300))
entities

In [ ]:
df = pd.DataFrame.from_dict(entities, orient='index')
# make all columns into list
df['entity'] = df.apply(lambda x: [e for e in x if e != 'str'], axis = 1)
# get rid of all none
df['entity'] = df['entity'].apply(lambda x: [e for e in x if e is not None])
df['entity'] 

In [ ]:
prob_df = df[['entity']]
# find where prob_df.entity is empty list
mask_empty = prob_df.entity.apply(lambda x: len(x) == 0 if type(x) == list else False)
mask_empty.value_counts()
# prob_df[mask_empty].shape

In [ ]:
prob_df.to_csv('../int_data/prob_df.csv', index = False)